# 02: All Contract Types — The Complete Reference

KindScript enforces 6 types of architectural checks. Each one catches a different class of violation:

| Contract | Error Code | What it catches |
|----------|------------|----------------|
| `noDependency` | KS70001 | Forbidden imports between layers |
| `mustImplement` | KS70002 | Missing interface implementations |
| `purity` | KS70003 | I/O imports in pure layers |
| `noCycles` | KS70004 | Circular dependencies between layers |
| `colocated` | KS70005 | Missing counterpart files (e.g., tests) |
| *existence* | KS70010 | Derived locations that don't exist on disk |

This notebook demonstrates each one with a **violation** and a **fix**.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

// Boilerplate for architecture.ts files (locate<T>() syntax)
const BOILERPLATE = `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  mustImplement?: [string, string][];
  purity?: string[];
  noCycles?: string[];
  colocated?: [string, string][];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}
`.trimStart();

const KINDSCRIPT_JSON = JSON.stringify({ definitions: ["architecture.ts"] }, null, 2);
const TSCONFIG = JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2);

function makeProject(): string {
  const dir = Deno.makeTempDirSync({ prefix: "ksc-contracts-" });
  Deno.writeTextFileSync(`${dir}/kindscript.json`, KINDSCRIPT_JSON);
  Deno.writeTextFileSync(`${dir}/tsconfig.json`, TSCONFIG);
  return dir;
}

console.log("Setup complete.");

---

## A. `noDependency` — Forbidden Imports (KS70001)

The most common contract. Forbids imports from one layer to another.

**Use case:** In Clean Architecture, the domain layer must not import from infrastructure.

```typescript
noDependency: [
  ["domain", "infrastructure"],  // domain cannot import from infrastructure
]
```

In [ ]:
const demo1 = makeProject();
Deno.mkdirSync(`${demo1}/src/domain`, { recursive: true });
Deno.mkdirSync(`${demo1}/src/infrastructure`, { recursive: true });

// Violation: domain imports from infrastructure
Deno.writeTextFileSync(`${demo1}/src/domain/service.ts`, `
import { Database } from '../infrastructure/database';

export class DomainService {
  private db = new Database();
  getAll(): string[] {
    return this.db.query('SELECT * FROM entities');
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo1}/src/infrastructure/database.ts`, `
export class Database {
  query(sql: string): string[] { return [sql]; }
}
`.trimStart());

Deno.writeTextFileSync(`${demo1}/architecture.ts`, BOILERPLATE + `
export interface CleanContext extends Kind<"CleanContext"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app = locate<CleanContext>("src", {
  domain: {},
  infrastructure: {},
});

export const contracts = defineContracts<CleanContext>({
  noDependency: [["domain", "infrastructure"]],
});
`);

console.log("=== Violation: domain imports from infrastructure ===");
await ksc("check", demo1);

In [ ]:
// Fix: domain defines its own interface, no infrastructure import
Deno.writeTextFileSync(`${demo1}/src/domain/service.ts`, `
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getAll(): string[] {
    return this.store.query('SELECT * FROM entities');
  }
}
`.trimStart());

console.log("=== Fixed: domain uses its own interface ===");
await ksc("check", demo1);

Deno.removeSync(demo1, { recursive: true });

---

## B. `purity` — No I/O in Pure Layers (KS70003)

Ensures a layer has no side effects — no `fs`, `http`, `net`, `child_process`, or any of Node's ~50 built-in I/O modules.

**Use case:** Domain logic should be pure. If it needs to read a file, it receives the data through a port.

```typescript
purity: ["domain"]
```

In [ ]:
const demo2 = makeProject();
Deno.mkdirSync(`${demo2}/src/domain`, { recursive: true });

// Violation: domain imports Node.js fs module
Deno.writeTextFileSync(`${demo2}/src/domain/service.ts`, `
import * as fs from 'fs';

export class DomainService {
  readData(): string {
    return fs.readFileSync('/tmp/data.txt', 'utf-8');
  }
}
`.trimStart());

Deno.writeTextFileSync(`${demo2}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  domain: DomainLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}

export const app = locate<AppContext>("src", {
  domain: {},
});

export const contracts = defineContracts<AppContext>({
  purity: ["domain"],
});
`);

console.log("=== Violation: domain imports 'fs' ===");
await ksc("check", demo2);

In [ ]:
// Fix: inject data instead of reading directly
Deno.writeTextFileSync(`${demo2}/src/domain/service.ts`, `
export interface DataReader {
  read(path: string): string;
}

export class DomainService {
  constructor(private reader: DataReader) {}
  readData(): string {
    return this.reader.read('/tmp/data.txt');
  }
}
`.trimStart());

console.log("=== Fixed: domain uses injected reader ===");
await ksc("check", demo2);

Deno.removeSync(demo2, { recursive: true });

---

## C. `mustImplement` — Every Port Needs an Adapter (KS70002)

Ensures every exported interface in one layer has a class that `implements` it in another.

**Use case:** In Hexagonal Architecture, every port interface needs an adapter.

```typescript
mustImplement: [["ports", "adapters"]]
```

In [ ]:
const demo3 = makeProject();
Deno.mkdirSync(`${demo3}/src/ports`, { recursive: true });
Deno.mkdirSync(`${demo3}/src/adapters`, { recursive: true });

// Port interface with no adapter
Deno.writeTextFileSync(`${demo3}/src/ports/repository.port.ts`, `
export interface RepositoryPort {
  save(entity: unknown): void;
  findAll(): unknown[];
}
`.trimStart());

Deno.writeTextFileSync(`${demo3}/src/adapters/placeholder.ts`, `
export const placeholder = true;
`.trimStart());

Deno.writeTextFileSync(`${demo3}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  ports: PortsLayer;
  adapters: AdaptersLayer;
}

export interface PortsLayer extends Kind<"PortsLayer"> {}
export interface AdaptersLayer extends Kind<"AdaptersLayer"> {}

export const app = locate<AppContext>("src", {
  ports: {},
  adapters: {},
});

export const contracts = defineContracts<AppContext>({
  mustImplement: [["ports", "adapters"]],
});
`);

console.log("=== Violation: RepositoryPort has no adapter ===");
await ksc("check", demo3);

In [ ]:
// Fix: add an adapter that implements the port
Deno.writeTextFileSync(`${demo3}/src/adapters/repository.adapter.ts`, `
import { RepositoryPort } from '../ports/repository.port';

export class InMemoryRepositoryAdapter implements RepositoryPort {
  private store: unknown[] = [];

  save(entity: unknown): void {
    this.store.push(entity);
  }

  findAll(): unknown[] {
    return [...this.store];
  }
}
`.trimStart());

console.log("=== Fixed: InMemoryRepositoryAdapter implements RepositoryPort ===");
await ksc("check", demo3);

Deno.removeSync(demo3, { recursive: true });

---

## D. `noCycles` — No Circular Dependencies (KS70004)

Detects circular dependency chains between layers.

```typescript
noCycles: ["domain", "infrastructure"]
```

In [ ]:
const demo4 = makeProject();
Deno.mkdirSync(`${demo4}/src/domain`, { recursive: true });
Deno.mkdirSync(`${demo4}/src/infrastructure`, { recursive: true });

// Cycle: domain → infrastructure AND infrastructure → domain
Deno.writeTextFileSync(`${demo4}/src/domain/service.ts`, `
import { Database } from '../infrastructure/database';

export class DomainService {
  private db = new Database();
  getData(): string[] { return this.db.query('SELECT *'); }
}
`.trimStart());

Deno.writeTextFileSync(`${demo4}/src/infrastructure/database.ts`, `
import { DomainService } from '../domain/service';

export class Database {
  private service = new DomainService();
  query(sql: string): string[] { return [sql]; }
}
`.trimStart());

Deno.writeTextFileSync(`${demo4}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app = locate<AppContext>("src", {
  domain: {},
  infrastructure: {},
});

export const contracts = defineContracts<AppContext>({
  noCycles: ["domain", "infrastructure"],
});
`);

console.log("=== Violation: domain <-> infrastructure cycle ===");
await ksc("check", demo4);

In [ ]:
// Fix: break the cycle — use dependency inversion
Deno.writeTextFileSync(`${demo4}/src/domain/service.ts`, `
export interface DataStore {
  query(sql: string): string[];
}

export class DomainService {
  constructor(private store: DataStore) {}
  getData(): string[] { return this.store.query('SELECT *'); }
}
`.trimStart());

Deno.writeTextFileSync(`${demo4}/src/infrastructure/database.ts`, `
import { DataStore } from '../domain/service';

export class Database implements DataStore {
  query(sql: string): string[] { return [sql]; }
}
`.trimStart());

console.log("=== Fixed: one-directional dependency (infra -> domain) ===");
await ksc("check", demo4);

Deno.removeSync(demo4, { recursive: true });

---

## E. `colocated` — Every File Needs a Counterpart (KS70005)

Ensures files in one directory have matching filenames in another.

**Use case:** "Every component has a test file."

```typescript
colocated: [["components", "tests"]]
```

In [ ]:
const demo5 = makeProject();
Deno.mkdirSync(`${demo5}/src/components`, { recursive: true });
Deno.mkdirSync(`${demo5}/src/tests`, { recursive: true });

// Two components, only one test
Deno.writeTextFileSync(`${demo5}/src/components/button.ts`, `
export function Button() { return 'button'; }
`.trimStart());

Deno.writeTextFileSync(`${demo5}/src/components/form.ts`, `
export function Form() { return 'form'; }
`.trimStart());

Deno.writeTextFileSync(`${demo5}/src/tests/button.ts`, `
import { Button } from '../components/button';
console.assert(Button() === 'button');
`.trimStart());

Deno.writeTextFileSync(`${demo5}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  components: ComponentsLayer;
  tests: TestsLayer;
}

export interface ComponentsLayer extends Kind<"ComponentsLayer"> {}
export interface TestsLayer extends Kind<"TestsLayer"> {}

export const app = locate<AppContext>("src", {
  components: {},
  tests: {},
});

export const contracts = defineContracts<AppContext>({
  colocated: [["components", "tests"]],
});
`);

console.log("=== Violation: form.ts has no counterpart test ===");
await ksc("check", demo5);

In [ ]:
// Fix: add the missing test
Deno.writeTextFileSync(`${demo5}/src/tests/form.ts`, `
import { Form } from '../components/form';
console.assert(Form() === 'form');
`.trimStart());

console.log("=== Fixed: form.ts now has a test ===");
await ksc("check", demo5);

Deno.removeSync(demo5, { recursive: true });

---

## F. Existence Checking — Missing Directories (KS70010)

When you use `locate<T>()`, KindScript derives directory paths from the root + member names. It then verifies those directories actually exist. This is automatic — no contract declaration needed.

**Use case:** You define structure in `architecture.ts` but haven't created the directories yet.

In [ ]:
const demo6 = makeProject();
// Only create domain — infrastructure is missing on disk
Deno.mkdirSync(`${demo6}/src/domain`, { recursive: true });

Deno.writeTextFileSync(`${demo6}/src/domain/entity.ts`, `
export interface Order { id: string; }
`.trimStart());

Deno.writeTextFileSync(`${demo6}/architecture.ts`, BOILERPLATE + `
export interface AppContext extends Kind<"AppContext"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app = locate<AppContext>("src", {
  domain: {},
  infrastructure: {},
});

export const contracts = defineContracts<AppContext>({});
`);

console.log("=== Violation: infrastructure directory doesn't exist ===");
await ksc("check", demo6);

In [ ]:
// Fix: create the missing directory (or use ksc scaffold)
Deno.mkdirSync(`${demo6}/src/infrastructure`, { recursive: true });
Deno.writeTextFileSync(`${demo6}/src/infrastructure/index.ts`, `
// Infrastructure adapter placeholder
`.trimStart());

console.log("=== Fixed: infrastructure directory created ===");
await ksc("check", demo6);

Deno.removeSync(demo6, { recursive: true });

The fix for existence violations is either:
- Create the directory manually
- Run `ksc scaffold --write` to create all missing directories at once

---

## Summary

| Contract | Code | Catches | Fix pattern |
|----------|------|---------|-------------|
| `noDependency` | KS70001 | Forbidden imports between layers | Dependency injection — define interfaces in the inner layer |
| `mustImplement` | KS70002 | Port interfaces without adapters | Add a class that `implements` the interface |
| `purity` | KS70003 | I/O imports (`fs`, `http`, etc.) in pure layers | Inject I/O through constructor |
| `noCycles` | KS70004 | Circular dependency chains | Break the cycle with interfaces (Dependency Inversion) |
| `colocated` | KS70005 | Missing counterpart files | Add the missing file |
| *existence* | KS70010 | Missing derived directories | Create directory or run `ksc scaffold` |

Contracts are combined in `defineContracts<T>()`:

```typescript
export const contracts = defineContracts<CleanContext>({
  noDependency: [
    ["domain", "infrastructure"],
    ["domain", "application"],
    ["application", "infrastructure"],
  ],
  purity: ["domain"],
});
```

**Next:** See **03-stdlib-and-ci.ipynb** for standard library packages and CI integration